# Homework 01. Hojune Lee (Student ID - 20161333)
CDC(Center for Disease Control and Prevention) released the information about PCR primers/probes to detect Covid-19. (See [this page](https://www.cdc.gov/coronavirus/2019-ncov/lab/rt-pcr-panel-primer-probes.html) for more information). We are curious how these primers/probes work.

If you need more info about "PCR-based diagnosis", see [this video](https://www.youtube.com/watch?v=fkUDu042xic).

## Data files
<ul>
    <li> The genome of Covid-19: '../data/2019nCoV_genomes.2020_02_03.fa'
    <li> The primers for Covid-19 detection: '../data/2019nCoV_primers.fa'
</ul>
    
## Procedures
<ol>
<li> Read 2019nCoV primers from a FASTA file (**see below**).
<li> Read 2019nCoV genomes from a FASTA file (**see below**).
<li> Find the position of primers (F, R) on each genome sequence.
<li> Calculate the length of PCR amplicons for Covid-19 diagnostics.
</ol>

## Questions
<ol>
    <li> What is the length of amplicons generated by N1/N2/N3 primers? Any variation among the genomes?
    <li> What is the sequence of amplicons?
    <li> Can these primers detect all Covid-19 genomes?
    <li> Can these primers detect MERS genomes? How about SARS genomes? (you can find those genomes under '../data' directory).
</ol>

In [ ]:
# The function to read sequences from a FASTA file
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

In [15]:
# Read genomes
filename_genomes = '../data/2019nCoV_genomes.2020_02_03.fa'
genome_list = read_fasta(filename_genomes)
for g_h, g_seq in genome_list.items():
    print(g_h.split()[0], g_seq[:25])

ENA|MN988668|MN988668.1 TTAAAGGTTTATACCTTCCCAGGTA
ENA|MN988669|MN988669.1 TTAAAGGTTTATACCTTCCCAGGTA
ENA|MN938384|MN938384.1 CAACCAACTTTCGATCTCTTGTAGA
ENA|MN975262|MN975262.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN996527|MN996527.1 AACCAACTTTCGATCTCTTGTAGAT
ENA|MN996528|MN996528.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN996529|MN996529.1 TACCTTCCCAGGTAACAAACCAACC
ENA|MN996530|MN996530.1 CCTTCCCAGGTAACAAACCAACCAA
ENA|MN996531|MN996531.1 ACCTTCCCAGGTAACAAACCAACCA
ENA|MN908947|MN908947.3 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN985325|MN985325.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN988713|MN988713.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN994467|MN994467.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN994468|MN994468.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN997409|MN997409.1 ATTAAAGGTTTATACCTTCCCAGGT


In [19]:
#HW 1-1
#Number of sequences

num_seq = len(genome_list)
print("Number of sequences:", num_seq)

Number of sequences: 15


In [51]:
#HW 1-2
#Length of sequences
length_of_sequences = []
for g_seq in genome_list.values():
    length_of_sequences.append(len(g_seq))
print("Length of sequences:", length_of_sequences)


Length of sequences: [29881, 29881, 29838, 29891, 29825, 29891, 29852, 29854, 29857, 29903, 29882, 29882, 29882, 29883, 29882]


In [57]:
#HW 1-3
#Number of identical sequences + their names

identical_sequences = []
name_of_sequences = list(genome_list.keys())
sequences_to_check = list(genome_list.values())

#erase a sequence from the list and see if the list has the same ones. 
while(len(sequences_to_check) > 1):
    current_sequence = sequences_to_check.pop(0)
    if current_sequence not in sequences_to_check:
        name_of_sequences.pop(0)
    else:
        templist = []
        templist.append(name_of_sequences.pop(0).split()[0])
        while current_sequence in sequences_to_check:
            idx = sequences_to_check.index(current_sequence)
            sequences_to_check.pop(idx)
            templist.append(name_of_sequences.pop(idx).split()[0])
        identical_sequences.append(templist)
        

for sequences in identical_sequences:
    print(sequences, "are identical with each other.")
print("number of identical sequences: {}".format(len(identical_sequences)))

['ENA|MN988668|MN988668.1', 'ENA|MN988669|MN988669.1'] are identical with each other.
number of identical sequences: 1


In [ ]:
# Read primers

filename_primers = '../data/2019nCoV_primers.fa'
primer_list = read_fasta(filename_primers)
for p_h, p_seq in primer_list.items():
    print(p_h, p_seq)

In [ ]:
# A function that consider IUPAC Nucleotide code when comparing codes
# See whether a probe which has codes such as R, Y and S is a part of a genome which has only A,T,G and C for its codes.

def IsProbe1(probe, genome):
	possible_probes = [probe]
	for idx in range(len(probe)):
		if probe[idx] == 'R':
			possible_probes = (" ".join(possible_probes).replace(probe[idx], 'A', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'G', 1)).split(" ")
		elif probe[idx] == 'Y':
			possible_probes = (" ".join(possible_probes).replace(probe[idx], 'C', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'T', 1)).split(" ")
		elif probe[idx] == 'S':
			possible_probes = (" ".join(possible_probes).replace(probe[idx], 'G', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'C', 1)).split(" ")
		elif probe[idx] == 'W':
			possible_probes = (" ".join(possible_probes).replace(probe[idx], 'A', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'T', 1)).split(" ")
		elif probe[idx] == 'K':
			possible_probes = (" ".join(possible_probes).replace(probe[idx], 'G', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'T', 1)).split(" ")
		elif probe[idx] == 'M':
			possible_probes = (" ".join(possible_probes).replace(probe[idx], 'A', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'C', 1)).split(" ")
		elif probe[idx] == 'B':
			possible_probes = (" ".join(possible_probes).replace(probe[idx], 'C', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'G', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'T', 1)).split(" ")
		elif probe[idx] == 'D':
			possible_probes = (" ".join(possible_probes).replace(probe[idx], 'A', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'G', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'T', 1)).split(" ")
		elif probe[idx] == 'H':
			possible_probes = (" ".join(possible_probes).replace(probe[idx], 'A', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'C', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'T', 1)).split(" ")
		elif probe[idx] == 'V':
			possible_probes = (" ".join(possible_probes).replace(probe[idx], 'A', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'C', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'G', 1)).split(" ")
		elif probe[idx] == 'N':
			possible_probes = (" ".join(possible_probes).replace(probe[idx], 'A', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'G', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'T', 1) + " " + (" ".join(possible_probes)).replace(probe[idx], 'C', 1)).split(" ")
	for possible_probe in possible_probes:
		if possible_probe in genome:
			return True
		else:
			continue
	return False

In [ ]:
# The fuction to get an amplicon from Primers F, R, P and a genome to be PCRed
# If they doesn't matches, return ''.
def primerFRP_to_amplicon(F_primer, R_primer, P_primer, genome):
    ap = ''
    if F_primer not in genome:
        return ap
    if revcomp(R_primer) not in genome:
        return ap
    if F_primer in genome and revcomp(R_primer) in genome:
        print(genome[:50])
        ap = genome[genome.index(F_primer):genome.index(revcomp(R_primer))] + revcomp(R_primer)
        return ap
    return ''